## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод

В данных почти нет пропусков. Пропущенные значения есть только в двух столбцах: с информацией о стаже работы и ежемесячном доходе. При этом количество ненулевых значений в этих двух столбцах одинаковое: скорее всего, пропуски находятся в одних и тех же строках. 

### Шаг 2. Предобработка данных

### Обработка пропусков

Изучим строки с пропущенными значениями. Попытаемся найти закономерности, объясняющие, почему пропуски образовались.

In [3]:
data[data['days_employed'].isna() == True].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


У людей с пропусками в данных о трудовом стаже и доходе нет ничего общего: они разного пола, возраста и уровня образования. Посчитаем количество строк, где пропущенные значения в этих двух столбцах не совпадают.

In [4]:
data[(data['days_employed'].isna() == True) & (data['total_income'].isna() != True)]['total_income'].count()

0

Действительно, пропуски совпадают во всей строках. Скорее всего, люди не захотели указывать эту информацию о себе при заполнении заявки.

Попробуем найти особенности, присущие тем, у кого пропущены значения в сведениях о стаже и доходе.

Сохраним таблицу, содержащую анкеты с пропущенными значениями.

In [5]:
data_null = data[data['days_employed'].isna() == True]

Проверим, расходятся ли у запонивших анкету не полностью уровень образования, возраст, пол и семейное положение с теми, кто дал полные сведения о себе.

In [6]:
data['education'].value_counts(normalize = True).head()

среднее                0.638792
высшее                 0.219187
СРЕДНЕЕ                0.035865
Среднее                0.033031
неоконченное высшее    0.031034
Name: education, dtype: float64

In [7]:
data_null['education'].value_counts(normalize = True).head()

среднее                0.647654
высшее                 0.228151
СРЕДНЕЕ                0.030819
Среднее                0.029899
неоконченное высшее    0.025299
Name: education, dtype: float64

In [8]:
data['family_status'].value_counts(normalize = True).head()

женат / замужем          0.575145
гражданский брак         0.194053
Не женат / не замужем    0.130685
в разводе                0.055517
вдовец / вдова           0.044599
Name: family_status, dtype: float64

In [9]:
data_null['family_status'].value_counts(normalize = True).head()

женат / замужем          0.568997
гражданский брак         0.203312
Не женат / не замужем    0.132475
в разводе                0.051518
вдовец / вдова           0.043698
Name: family_status, dtype: float64

In [10]:
data['gender'].value_counts(normalize = True).head()

F      0.661370
M      0.338583
XNA    0.000046
Name: gender, dtype: float64

In [11]:
data_null['gender'].value_counts(normalize = True).head()

F    0.682613
M    0.317387
Name: gender, dtype: float64

In [12]:
data['income_type'].value_counts(normalize = True).head()

сотрудник      0.516562
компаньон      0.236237
пенсионер      0.179141
госслужащий    0.067782
безработный    0.000093
Name: income_type, dtype: float64

In [13]:
data_null['income_type'].value_counts(normalize = True).head()

сотрудник          0.508280
компаньон          0.233671
пенсионер          0.189972
госслужащий        0.067617
предприниматель    0.000460
Name: income_type, dtype: float64

Объединим заполнивших в возрастные группы, чтобы было удобнее считать и соотносить их данные.

In [14]:
def age_group(age):
    #функция принимает число (возраст) 
    #и возвращает промежуток, 
    #в котором это число находится
    if 18 <= age <= 25:
        return '18-25'
    if 26 <= age <= 35:
        return '26-35'
    if 36 <= age <= 45:
        return '36-45'
    if 46 <= age <= 55:
        return '46-55'
    if 56 <= age <= 64:
        return '56-64'
    if age >= 65:
        return '65+'

Проверим работу функции.

In [15]:
age_group(47)

'46-55'

In [16]:
age_group(77)

'65+'

In [17]:
age_group(21)

'18-25'

Создадим в таблице новый столбец с информацией о возрастной группе.

In [18]:
data['age_group'] = data['dob_years'].apply(age_group)

Посмотрим, есть ли расхождения среди заполнивших и не заполнивших данные о стаже и доходе по возрастным группам.

In [78]:
data['age_group'].value_counts(normalize = True)

36-45   26.6%
26-35   24.9%
46-55   22.4%
56-64   16.1%
18-25    5.7%
65+      4.2%
Name: age_group, dtype: float64

In [79]:
data_null = data[data['days_employed'].isna() == True]
data_null['age_group'].value_counts(normalize = True)

Series([], Name: age_group, dtype: float64)

Расхождения не найдены ни по одному из признаков. Люди, не указывающие сведения о своём стаже и доходе, никак не связаны между собой.
Вполне возможно, произошла какая-то специфическая ошибка при выгрузке данных. Но скорее всего люди просто не захотели их заполнять (на это указывает полное совпадение пропусков в данных о стаже и доходе).

Пропущенные данные имеют количественный вид. Учитывая, что никаких закономерностей между строками с пропущенными и заполненными данными не найдено, заполним их средними или медианными значениями.

Для начала проверим минимальное и максимальное значение, чтобы затем принять решение о том, какими значениями заполнять пропуски: средними или медианой.

In [21]:
data['days_employed'].max()

401755.40047533

In [22]:
data['days_employed'].min()

-18388.949900568383

В данных есть ошибка: у некоторых людей записано отрицательное число в данных о стаже. Стаж не может быть отрицательным — необходимо выяснить, в чём ошибка.
Кроме того, стаж явно не может составлять 401755 дней (больше тысячи лет).

Посмотрим, сколько строк в столбце со стажем имеют отрицательные значения, и к какой категории занятости они относятся.

In [23]:
data[data['days_employed'] < 0]['income_type'].unique()

array(['сотрудник', 'компаньон', 'госслужащий', 'студент',
       'предприниматель', 'в декрете'], dtype=object)

In [24]:
data[data['days_employed'] > 0]['income_type'].unique()

array(['пенсионер', 'безработный'], dtype=object)

In [25]:
data[data['days_employed'] == 0]['income_type'].unique()

array([], dtype=object)

Все, кто работают на данный момент, имеют отрицательное значение в графе о стаже. Пенсионеры и безработные — положительные значения.

В таком случае корректнее будет посчитать средние значения для каждого типа занятости.

Составим таблицу средний значений стажа для каждой категории занятости.

In [26]:
data_mean_days = data.groupby('income_type').agg({'days_employed' : 'mean'})
data_mean_days

,days_employed
income_type,
безработный,366413.652744
в декрете,-3296.759962
госслужащий,-3399.896902
компаньон,-2111.524398
пенсионер,365003.491245
предприниматель,-520.848083
сотрудник,-2326.499216
студент,-578.751554


Заполним пустые значения соответствующих категорий средними значениями.

In [27]:
for income_type in data_mean_days.index:
    data.loc[
        data['income_type'] == income_type, 'days_employed'
    ] = data.loc[
        data['income_type'] == income_type, 'days_employed'
    ].fillna(data_mean_days.loc[income_type, 'days_employed'])

Проверим, заполнены ли данные.

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
age_group           21424 non-null object
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


Теперь по такой же схеме заполним пропуски в данных о среднем доходе. Сделаем это так же: для каждой категории занятости.

In [30]:
income_mean = data.groupby('income_type').agg({'total_income' : 'mean'})
income_mean

,total_income
income_type,
безработный,131339.751676
в декрете,53829.130729
госслужащий,170898.309923
компаньон,202417.461462
пенсионер,137127.465690
предприниматель,499163.144947
сотрудник,161380.260488
студент,98201.625314


Заполним пустые значения данными из таблицы выше.

In [31]:
for income_type in income_mean.index:
    data.loc[
        data['income_type'] == income_type, 'total_income'
    ] = data.loc[
        data['income_type'] == income_type, 'total_income'
    ].fillna(income_mean.loc[income_type, 'total_income'])

Убедимся, что пропусков не осталось.

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
age_group           21424 non-null object
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


Также посмотрим, сколько пропусков в столбце с информацией о поле.

In [33]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Чтобы заменить пропущенное значение на один из двух полов, посмотрим на строку с ним.

In [34]:
data[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости,18-25


Посмотрим, кто чаще берёт кредит на покупку недвижимости: мужчины или женщины.

In [35]:
data[data['purpose'] == 'покупка недвижимости']['gender'].value_counts()

F      395
M      228
XNA      1
Name: gender, dtype: int64

Такой кредит чаще берут женщины. Заменим пустое значение на женский пол.

In [36]:
data['gender'] = data['gender'].replace('XNA', 'F')

Проверим, что замена произведена успешно.

In [37]:
data['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

Проверим столбец с данными о возрасте.

In [38]:
data[data['dob_years'] < 18]['dob_years'].value_counts()

0    101
Name: dob_years, dtype: int64

101 человек имеет 0 в столбце с возрастом. Считать среднее будет также для каждой категории занятости отдельно, ведь одно среднее не подойдёт и студентам, и пенсионерам.

In [39]:
dob_years_mean = data.groupby('income_type').agg({'dob_years' : 'mean'})
dob_years_mean

,dob_years
income_type,
безработный,38.000000
в декрете,39.000000
госслужащий,40.636737
компаньон,39.697542
пенсионер,59.063019
предприниматель,42.500000
сотрудник,39.821027
студент,22.000000


Округлим значения, чтобы сохранить целочисленный тип данных и избежать анкеты с 39.697542 лет в данных о возрасте.

In [40]:
dob_years_mean['dob_years'] = dob_years_mean['dob_years'].round().astype(int)
dob_years_mean

,dob_years
income_type,
безработный,38
в декрете,39
госслужащий,41
компаньон,40
пенсионер,59
предприниматель,42
сотрудник,40
студент,22


In [41]:
for income_type in dob_years_mean.index:
    data.loc[
        data['income_type'] == income_type, 'dob_years'
    ] = data.loc[
        data['income_type'] == income_type, 'dob_years'
    ].replace(0, dob_years_mean.loc[income_type, 'dob_years'])

Проверим столбец с возрастом на наличие нулевых значений.

In [42]:
data[data['dob_years'] < 18]['dob_years'].count()

0

Обновим столбец с возрастными группами.

In [43]:
data['age_group'] = data['dob_years'].apply(age_group)

Убедимся, что пропусков (даже неявных) больше нет ни в одном из столбцов.

In [44]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [45]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [46]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В данных есть артефакты: -1 ребенок и 20 детей. Скорее всего, это связано с неправильным вводом данных оператором: -1 соответствует 1, а 20 — двум детям. Сразу избавимся от ошибок.

In [47]:
data['children'] = data['children'].replace(20, 2)
data['children'] = data['children'].replace(-1, 1)

Проверим данные.

In [48]:
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [49]:
data['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

### Вывод

В начале казалось, что пропуски есть только в данных о трудовом стаже и доходе. В итоге пропущенные значения нашлись ещё и в данных о возрасте и поле. Также вскрылось несколько других проблем в данных, с которыми тоже придётся разобраться.
Пропуски в данных о стаже и доходе полностью совпадают по строкам. Скорее всего, некоторые люди предпочитают не делиться информацией о стаже и доходе (может быть, потому что они слишком низкие).
Остальные пропуски (возраст и пол), могут быть связаны как с ошибкой выгрузки, так и с нежеланием человека заполнять эти данные о себе.

### Замена типа данных

Стаж (в днях) и доход (в рублях) сейчас имеют вещественный вид с большим числом знаков после запятой. Округлим и сделаем значения целочисленными, чтобы не было нескольких знаков после запятой в информации о длительности стажа или доходах.

In [50]:
data['days_employed'] = data['days_employed'].round().astype(int)

In [51]:
data['total_income'] = data['total_income'].round().astype(int)

Проверим данные.

In [52]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
age_group           21525 non-null object
dtypes: int64(7), object(6)
memory usage: 2.1+ MB


### Вывод

В целом проблем с типом данных в таблице было немного. Нужно было лишь перевести столбцы с днями и рублями в целочисленный вид, что мы и сделали.

### Обработка дубликатов

Сначала сделаем буквы во всех столбцах с нечисловыми значениями строчными (кроме столбца с типом занятости: про него мы уже знаем, что все значения записаны строчными буквами).

In [53]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['purpose'] = data['purpose'].str.lower()

Проверим, например, данные об образовании.

In [54]:
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Посмотрим, есть ли в таблице полностью дублирующиеся строки.

In [55]:
data.duplicated().sum()

71

В таблице 71 дубликат. Нельзя сказать наверняка, дублирующиеся это строки, или данные нескольких людей полностью совпали между собой (что возможно после заполнения пропусков средними). Тем не менее дубликатов немного, поэтому ими можно принебречь и удалить.

In [56]:
data = data.drop_duplicates().reset_index(drop=True)

Проверим таблицу.

In [57]:
data.duplicated().sum()

0

### Вывод

В таблице было несколько дубликатов, которые могли появиться при выгрузке или при заполнении пропусков средними значениями. Относительно всего массива данных их было немного — их можно было удалить без большого ущерба для анализа.

### Лемматизация

Лемматизируем данные о цели кредита, чтобы разделить их на несколько отдельных групп.

In [58]:
from pymystem3 import Mystem
m = Mystem()

In [59]:
data['purpose_lemmas'] = data['purpose'].apply(m.lemmatize)

Посмотрим на самые частые слова, чтобы определить список основных целей кредита.

In [60]:
from collections import Counter
Counter(data['purpose_lemmas'].sum())

Counter({'покупка': 5897,
         ' ': 33570,
         'жилье': 4460,
         '\n': 21454,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 478,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'подержанный': 486,
         'получение': 1314,
         'высокий': 1374,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607})

Основных целей 5: автомобиль, свадьба, жильё, недвижимость, образование. Жильё и недвижимость можно объединить в одну группу.

### Вывод

Лемматизация позволила нам выделить основные цели, для которых люди берут кредит. Это покупка (иногда ремонт) недвижимости, жилья или автомобиля, оплата образования или организация свадьбы.

### Категоризация данных

С помощью лемматизации выделим основные цели кредита. Это позволит затем посчитать вероятность его возврата в зависимости от цели.

In [61]:
def purpose_pure(lemmas_list):
    if 'автомобиль' in lemmas_list:
        return 'автомобиль'
    if 'свадьба' in lemmas_list:
        return 'свадьба'
    if ('недвижимость' in lemmas_list) or ('жилье' in lemmas_list):
        return 'недвижимость'
    if 'образование' in lemmas_list:
        return 'образование'

Создадим отдельный столбец с информацией о конкретной цели кредита.

In [62]:
data['purpose_pure'] = data['purpose_lemmas'].apply(purpose_pure)

Проверим данные.

In [63]:
data['purpose_pure'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_pure, dtype: int64

Мы разделили все цели кредита на 4 основные: покупка недвижимости, приобретение автомобиля, организация свадьбы и оплата образования.

Теперь разделим на две явные категории данные о семейном положении. Создадим «словарь» данных о семейном положении, чтобы затем использовать его в подсчётах.

In [64]:
family_status_dict = data[['family_status', 'family_status_id']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Напишем функцию, которая принимает количество детей (по «словарю») и возвращает информацию о наличии или отсутствии детей у человека.

In [65]:
def children_TF(children):
    if children == 0:
        return 'нет детей'
    return 'есть дети'

Создадим новый столбец с информацией о наличии детей.

In [66]:
data['children_TF'] = data['children'].apply(children_TF)

Теперь получим аналогичную информацию для людей, у которых есть или отсутствует партнёр. Для этого напишем функцию, которая принимает информацию о семейном положении, а возвращает информацию о наличии или отсутствии пары.

In [67]:
def lonely_TF(family_status_id):
    if family_status_id == 0 or family_status_id == 1:
        return 'есть пара'
    return 'одинокий'

Создадим новый столбец с информацией о наличии партнёра.

In [68]:
data['lonely_TF'] = data['family_status_id'].apply(lonely_TF)

Поделим всех людей на две равные группы по доходу. Для этого нужно найти медиану.

In [69]:
total_income_med = data['total_income'].median()

Теперь напимем функцию, которая принимает доход человека, а возвращает информацию о том, выше она или ниже, чем медиана.

In [70]:
def income_cat(income):
    if income < total_income_med:
        return 'ниже среднего'
    return 'выше среднего'

Создадим новый столбец с информацией о доходной группе.

In [71]:
data['income_cat'] = data['total_income'].apply(income_cat)

### Вывод

С помощью категоризации мы разделили все анкеты на несколько (не более 4) групп, которые помогут более просто проанализировать данные.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Создадим таблицу с информацией о среднем числе людей, имеющих долг по кредиту, в каждой из двух групп людей: с детьми и без.

Для начала отформатируем вывод вещественных чисел для более понятного вида результата.

In [72]:
pd.options.display.float_format = '{:.1%}'.format

In [73]:
children_debt_stats = data.groupby('children_TF').agg({'debt' : 'mean'})
children_debt_stats = children_debt_stats['debt']
children_debt_stats

children_TF
есть дети   9.2%
нет детей   7.5%
Name: debt, dtype: float64

### Вывод

Люди без детей имеют меньше долгов по кредитам — на 1,7 процентных пункта. Видимо, люди с детьми в целом чаще берут кредит, но им и сложнее его возвращать из-за дополнительных трат на иждивенцев.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Посчитаем процент людей, имеющих долги по кредитам, в каждой из групп.

In [74]:
family_stats = data.groupby('lonely_TF').agg({'debt' : 'mean'})
family_stats = family_stats['debt']
family_stats

lonely_TF
есть пара   8.0%
одинокий    8.5%
Name: debt, dtype: float64

### Вывод

Люди, имеющие партнёра, имеют меньше долгов по кредитам — но разница небольшая: всего 0,5 процентных пункта. Скорее всего, наличие партнёра не влияет на возврат кредита в срок.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Посчитаем процент людей, имеющих долги по кредитам, в каждой из групп.

In [75]:
income_stats = data.groupby('income_cat').agg({'debt' : 'mean'})
income_stats = income_stats['debt']
income_stats

income_cat
выше среднего   7.8%
ниже среднего   8.4%
Name: debt, dtype: float64

Так же попробуем посмотреть крайние значения: для людей с доходом меньше 100 тысяч и больше 500 тысяч.

In [76]:
def income_check(income):
    if income < 100000:
        return 'меньше 100 тысяч'
    if income > 500000:
        return 'больше 500 тысяч'
data['income_check'] = data['total_income'].apply(income_check)
income_stats_1 = data.groupby('income_check').agg({'debt' : 'mean'})
income_stats_1 = income_stats_1['debt']
income_stats_1

income_check
больше 500 тысяч   6.3%
меньше 100 тысяч   7.9%
Name: debt, dtype: float64

### Вывод

Люди с более высоким доходом имеют меньше долгов по кредитам. Это логично: у них больше денег, чтобы расплачиваться по долгам. Но разница тоже не очень большая: 0,6 процентных пункта. Чем доход дальше от медианы, тем больше будет разница. При рассмотрении групп с доходом меньше 100 тысяч и больше 500 тысяч разница увеличивается до 1,6 процентных пункта.

- Как разные цели кредита влияют на его возврат в срок?

In [77]:
purpose_stats = data.groupby('purpose_pure').agg({'debt' : 'mean'})
purpose_stats = purpose_stats['debt']
purpose_stats

purpose_pure
автомобиль     9.4%
недвижимость   7.2%
образование    9.2%
свадьба        8.0%
Name: debt, dtype: float64

### Вывод

Меньше всего долгов по кредитам у людей, которые берут его на покупку недвижимости. Причин может быть несколько: 
— такие люди в целом более ответственно относятся к финансам (т.к. суммы кредита самые большие), 
— у таких людей чаще кредит на недвижимость — первый (недвижимость дорогая, на другие покупки денег хватало, и кредит не требовался. Соотвественно, нет и долгов по нему),
— самая большая ответственность за невозврат кредита (недвижимость могут конфисковать в пользу кредитной организации).

На втором месте свадьба. Людей, имеющих долги, в этой категории на 0,8 процентных пункта больше. Кредит на свадьбу берут более молодые люди, у них в целом более чистая кредитная история. Тем не менее организация свадьбы стоит куда меньше, чем объект недвижимости. Возможно, некоторые люди именно из-за долгов по другим кредитам и решили взять ещё один кредит не свадьбу, не имея возможности накопить на неё. Это могут быть люди из молодых семей, которые уже взяли кредит на квартиру (например, воспользовавшись льготной ипотекой).

Люди, берущие деньги на покупку автомобиля или оплату образования, имеют больше всех долгов. Должников среди них на 1,2-1,3 процентных пункта больше, чем среди людей, берущих кредит на свадьбу, и на 2 процентных пункта больше, чем среди тех, кто берёт ипотеку. Скорее всего, это связано со следующими причинами:
— чаще всего кредит на образование берут или родители будущих студентов, или взрослые люди, которые захотели повысить квалификацию или совсем изменить направление работы. Возможно, и те, и другие испытывают больше трудностей с возвратом прошлых кредитов (на жильё или автомобиль, особенно если кредит берут родители на образование ребёнка). Опять-таки, если кредит на образование взял для себя взрослый человек, он, скорее всего, недоволен своим нынешним уровнем достатка. Кроме того, ответственность не так велика, потому что конфисковать у человека образование не получится. То есть в случае невозврата потери не так высоки, как при покупке имущества.
— автомобиль стоит дешевле, чем квартира (в среднем — ведь человек, живущий в квартире за 5 миллионов рублей, не станет покупать себе машину за 10 миллионов), ответственность в то же время не ниже (его тоже могут забрать за неуплату долга). И всё же люди берут кредит на автомобиль, имея другие долги. Это тоже могут быть молодые семьи, которые имеют долги по ипотеке (а ипотеку берут на несколько десятков лет, поэтому долг будет ещё долго), или люди с не очень высоким доходом, у которых уже есть кредиты за технику для дома или ремонт.

### Шаг 4. Общий вывод

В процессе предобработки данных всплыло несколько проблем: отрицательные значения стажа у людей, работающих на данный момент, и огромные числа у тех, кто не работает (безработных и пенсионеров), пропуски в данных о стаже и доходе, нулевые значения в данных о возрасте, одно нулевое в данных о поле. Пропуски и нули скорее всего связаны с нежеланием заполняющих делиться информацией о себе. Артефакты в данных о стаже имеют более интересную природу. Есть несколько гипотез:
1. Данные о стаже работающих людей сделаны отрицательными намеренно, чтобы отделить от неработающих. Числа в них — действительно количество дней трудового стажа.
2. Средний показатель «стажа» безработных — чуть 366 тысяч, пенсионеров — почти ровно 365 тысяч. Оба эти числа похожи на количество дней в году. Возможно, эти числа означают количество дней с потери работы (почему-то умноженное на тысячу). Это имеет смысл, потому что после года без работы у людей могут кончиться сбережения, и они могут задуматься о том, чтобы взять кредит на покупку чего-либо.
3. Возможно, такой вид записи сделан специально для внутреннего банковского калькулятора, который считает вероятность того, что человек НЕ вернёт долг. В таком случае чем больше у человека постоянный стаж работы, тем шанс невозврата ниже — число вычитается из общей формулы. Если он безработный — число прибавляется, предварительно умноженное на тысячу для увеличения веса, ведь отсутствие работы существенно повышает риск невозврата кредита в срок (или невозврата вообще).

На возврат кредита в срок больше всего влияют наличие детей, уровень дохода, а также цель, для которой кредит берут. В то же время наличие партнёра, судя по всему, почти не влияет на возврат долга по кредиту.